# Information Retrieval Coursework (STW7071CEM)

Task 1:
Create a vertical search engine comparable to Google Scholar, but specialized in retrieving just papers/books published by a member of Coventry University's Research Centre for Intelligent Healthcare (RCIH).


## Package Installs

Some packages required for this task are by default not included in the jupyter notebooks package list. A pip-install of these packages is required. 

In [3]:
!pip install scrapy
!pip install requests
!pip install BeautifulSoup4
!pip install nltk
!pip install gensim
!pip install xgboost
!pip install pandastable
!pip install pandas

You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Aayush\EDU_SOFT\IR assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
# nltk.download('omw-1.4');
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

# 1. Crawler Component

In [5]:
# Define Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

# Define profile URL format. This was obtained by manually examining the profile pages
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [7]:
def get_maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = get_maximum_page()
mx

38

In [8]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # Check department
        #print(span.text)
        if span.text == str('Centre for Intelligent Healthcare'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
     database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
     database.to_csv('database.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    
#     new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
#     page = requests.get(new_url)
#     soup = BeautifulSoup(page.content, "html.parser")
#     results = soup.find(id="main-content")
#     papers = results.find_all("li", class_="list-result-item")
    
    
#     for paper in papers:
#         title = paper.find('h3', class_='title').find('span')
#         author = paper.find('a', class_='link person').find('span')
#         date = paper.find('span', class_="date")
#         link = paper.find('h3', class_='title').find('a', href=True)['href']
        
        

    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")

    for paper in papers:
        title = paper.find('h3', class_='title')
        if title is not None:
            title_span = title.find('span')
            title_text = title_span.text if title_span is not None else "N/A"
        else:
            title_text = "N/A"

        author = paper.find('a', class_='link person')
        print(author)
        if author == None or author == 'None':
            continue
        else:
            author_span = author.find('span')
            author_text = author_span.text if author_span is not None else "N/A"

        date = paper.find('span', class_="date")
        date_text = date.text if date is not None else "N/A"

        link = paper.find('h3', class_='title')
        link_href = link.find('a', href=True)['href'] if link is not None else "N/A"
        print(link_href)
    
    # Rest of your code for appending the data to DataFrame or performing other operations

        # Read the existing database.csv file
        opening = pd.read_csv('database.csv', index_col="Unnamed: 0")

        # Create a new DataFrame with the data to be appended
        new_row = pd.DataFrame({'Title': [title.text],
                                'Author': [author.text],
                                'Published': [date.text],
                                'Link': [link_href]})

        # Concatenate the existing DataFrame and the new row DataFrame
        opening = pd.concat([opening, new_row], ignore_index=True)

        # Save the updated DataFrame to database.csv
        opening.to_csv('database.csv')

                

In [9]:
## Scrape function
def scrape(mx):
    df = update_csv('database.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [10]:
create_csv()
update_csv(database='database.csv')

%time scrape(mx) 

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/optimizing-rapid-prototype-development-through-femtosecond-laser-
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/monitoring-symptoms-of-infectious-diseases-perspectives-for-print
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/development-of-active-centrifugal-pump-for-microfluidic-cd-platfo
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/high-efficiency-small-sample-microparticle-fractionation-on-a-

In [11]:
sample_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'})
sample_db
print(f'{sample_db.shape[0]} records were scraped')

139 records were scraped


# 2. Indexing Component

In [12]:
scraped_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_db.head()

,SN,Title,Author,Published,Link
0,0,Optimizing Rapid Prototype Development Through...,"Albagdady, A.",1 Oct 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Monitoring Symptoms of Infectious Diseases: Pe...,"Albagdady, A.",27 May 2021,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Development of Active Centrifugal Pump for Mic...,"Albagdady, A.",27 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
3,3,High-Efficiency Small Sample Microparticle Fra...,"Albagdady, A.",30 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Enhanced inertial focusing of microparticles a...,"Albagdady, A.",18 Jun 2019,https://pureportal.coventry.ac.uk/en/publicati...


In [13]:
sample_db.head(7)

,SN,Title,Author,Published,Link
0,0,Optimizing Rapid Prototype Development Through...,"Albagdady, A.",1 Oct 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Monitoring Symptoms of Infectious Diseases: Pe...,"Albagdady, A.",27 May 2021,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Development of Active Centrifugal Pump for Mic...,"Albagdady, A.",27 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
3,3,High-Efficiency Small Sample Microparticle Fra...,"Albagdady, A.",30 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Enhanced inertial focusing of microparticles a...,"Albagdady, A.",18 Jun 2019,https://pureportal.coventry.ac.uk/en/publicati...
5,5,Sheath-less high throughput inertial separatio...,"Albagdady, A.",18 Dec 2019,https://pureportal.coventry.ac.uk/en/publicati...
6,6,New generation of spinning systems for robust ...,"Albagdady, A.",26 Jul 2018,https://pureportal.coventry.ac.uk/en/publicati...


In [14]:
single_row = scraped_db.loc[1,:].copy()
single_row

SN                                                           1
Title        Monitoring Symptoms of Infectious Diseases: Pe...
Author                                           Albagdady, A.
Published                                          27 May 2021
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 1, dtype: object

## 2.1 Preprocess Text

In [15]:
# Remove stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('','',string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [16]:
# Sample title
single_row['Title']

'Monitoring Symptoms of Infectious Diseases: Perspectives for Printed Wearable Sensors'

In [17]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

'monitoring symptom infectious disease perspective print wearable sensor '

In [18]:
# Demonstration of lematization

lematize(tp1(single_row['Title']))
#lematize(single_row['Title'])

'monitoring symptom infectious disease perspective print wearable sensor '

#### Unprocessed

In [19]:
sample_db['Title'].iloc[5]

'Sheath-less high throughput inertial separation of small microparticles in spiral microchannels with trapezoidal cross-section'

#### Processed

In [20]:
scraped_db['Title'].iloc[5]

'Sheath-less high throughput inertial separation of small microparticles in spiral microchannels with trapezoidal cross-section'

### 2.1.1 Preprocess entire dataframe

In [21]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

,SN,Title,Author,Published,Link
0,0,optimize rapid prototype development femtoseco...,"albagdady, a.",1 Oct 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,monitoring symptom infectious disease perspect...,"albagdady, a.",27 May 2021,https://pureportal.coventry.ac.uk/en/publicati...
2,2,development active centrifugal pump microfluid...,"albagdady, a.",27 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
3,3,highefficiency small sample microparticle frac...,"albagdady, a.",30 Jan 2020,https://pureportal.coventry.ac.uk/en/publicati...
4,4,enhance inertial focus microparticles cell int...,"albagdady, a.",18 Jun 2019,https://pureportal.coventry.ac.uk/en/publicati...


## 2.2 Index Construction

In [22]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('=====================================================================')
print('Sample index')
print(example)

SN                                                           0
Title        optimize rapid prototype development femtoseco...
Author                                           albagdady, a.
Published                                           1 Oct 2023
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 0, dtype: object
Sample index
{'optimize': [0]}


In [23]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})
#print(indx)

In [24]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

In [25]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [26]:
len(data)

715

In [27]:
data

{'185': [59],
 '2': [134],
 '2005–2008': [136],
 '2015': [138],
 '2018': [138],
 '2023': [10],
 '21st': [101],
 '3rs': [87],
 '3′utr': [59],
 '7': [91],
 'aankomende': [116],
 'ablation': [0],
 'abnormality': [41],
 'acceptance': [107],
 'accord': [66],
 'accuracy': [9, 30],
 'across': [117],
 'activation': [71],
 'active': [2, 6, 56],
 'adaptive': [68],
 'address': [122, 124],
 'adhd': [130, 131],
 'adjunct': [126],
 'adolescence': [109],
 'adolescent': [108, 110, 117, 125],
 'adult': [49, 84, 90, 92, 101, 108],
 'advance': [31],
 'age': [13, 29, 58, 61, 69, 70, 84, 86, 98, 100],
 'ageassociated': [66, 103],
 'agerelated': [32, 97],
 'aidriven': [17, 114],
 'al': [115],
 'alteration': [133],
 'amplitude': [51],
 'analysis': [14, 23, 51, 52, 54, 62, 63, 84, 111],
 'animal': [76],
 'anklebrachial': [43],
 'antigrippes': [85],
 'anxiety': [111],
 'application': [14, 23, 24, 108],
 'apply': [54, 134],
 'approach': [36, 61, 106, 125],
 'appropriate': [57],
 'arterial': [16, 18, 19, 30, 43,

## 3.  Query Processor

In [28]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
#demonstrate_query_processing()

### 3.1.  Split Query into individual terms

In [29]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms:  health


Processed Search Query: health 
Split Query: ['health']


### 3.2.  Boolean Functionalities

In [30]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

### 3.3. Search Engine Function

In [31]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [32]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query:  health


,SN,Title,Author,Published,Link
0,42,negotiate commission pathway successful implem...,"allen, j.",6 Sept 2019,https://pureportal.coventry.ac.uk/en/publicati...
1,117,mobile phone intervention improve obesityrelat...,"bul, k.",2 Mar 2020,https://pureportal.coventry.ac.uk/en/publicati...
2,118,development exploitation digital health soluti...,"bul, k.",9 Dec 2020,https://pureportal.coventry.ac.uk/en/publicati...
3,119,board game health systematic literature review...,"bul, k.",8 Apr 2019,https://pureportal.coventry.ac.uk/en/publicati...


In [33]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            #print(printout['Title'])
            #print(printout['Author'])
            #print(printout['Published'])
            #print(printout['Link'])
            #print('')

In [34]:
scraped_db['Author'].iloc[24]

'allen, j.'

In [35]:
final_engine(test_search_engine())

Enter your search query:  health


,SN,Title,Author,Published,Link
0,42,negotiate commission pathway successful implem...,"allen, j.",6 Sept 2019,https://pureportal.coventry.ac.uk/en/publicati...
1,117,mobile phone intervention improve obesityrelat...,"bul, k.",2 Mar 2020,https://pureportal.coventry.ac.uk/en/publicati...
2,118,development exploitation digital health soluti...,"bul, k.",9 Dec 2020,https://pureportal.coventry.ac.uk/en/publicati...
3,119,board game health systematic literature review...,"bul, k.",8 Apr 2019,https://pureportal.coventry.ac.uk/en/publicati...


## 4. Schedule Crawler for every week

To demonstrate a weekly scheduled crawling, the following parameters are defined:




In [ ]:
days = 0
interval = 7
while days <= 1:
    scrape(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/optimizing-rapid-prototype-development-through-femtosecond-laser-
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/monitoring-symptoms-of-infectious-diseases-perspectives-for-print
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/development-of-active-centrifugal-pump-for-microfluidic-cd-platfo
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/ahmed-albagdady" rel="Person"><span>Albagdady, A.</span></a>
https://pureportal.coventry.ac.uk/en/publications/high-efficiency-small-sample-microparticle-fractionation-on-a-

# 5. GUI

In [61]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')


def new_gui(image1):
    window = Tk()
    window.configure(bg='#EEEE63')
    window.title("Coventry University Search Engine")
    window.geometry('1100x600')
    
    lbl = Label(window, text="                Coventry Search Engine             ",bg="#337ab7", font=("Arial Bold", 40), padx=50, pady=50)
    lbl.grid(column=1, row=0)
    
    
    # img = ImageTk.PhotoImage(image1)
    
    # lbl3 = Label(image=img)
    # lbl3.image = img
    # lbl3.grid(column=1, row=3, padx=5, pady=5)
    
    
    
    query = Entry(window,width=80)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=400)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Do you want to quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search",bg="#B3B3EE", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [62]:
new_gui(resized_image1)